In [7]:
import os
import pydicom
import numpy as np
import torch 
import glob
import sys
import time 

import matplotlib.pyplot as plt
import matplotlib.path as mplPath

import pylab as pl

from threading import Timer

from collections import Counter
from collections import defaultdict

## Inspect a folder

In [8]:
class Folder(object): 
    
    def __init__(self, root, subject_substr=""): 
        
        if "\\" in root: 
            raise Exception("Please replace the backslashes in your path with forwardslashes.")
        self.root = root
        self.subfolders = self.get_subjects(subject_substr)
        
        self.walk_history = []
        self.walk_history.append(self.subfolders)

        self.walk_history_lost = []
        self.walk_history_lost.append([])
        
     
    def get_folders_files(self, path):
        for root, folders, files in os.walk(path):
            break   
        
        return folders, files
        
    def get_subjects(self, subject_substr):         
        subjects, _ = self.get_folders_files(self.root)
        
        return [s for s in subjects if subject_substr in s]

    def down(self, equals=[], contains=[], does_not_contain=[], print_absent=False): 
        if not isinstance(equals, list) or not isinstance(contains, list) or not isinstance(does_not_contain, list):
            raise Exception("Please enter a list with strings instead of a string.")
        
        print(f"\nSubfolder equals {str(equals)},   contains {str(contains)},   does not contain {str(does_not_contain)}")
        subfolders_present = []
        subfolders_absent = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            
            folders, files = self.get_folders_files(path)
            
            for string in equals: 
                folders = [f for f in folders if string.lower() == f.lower()]
            
            for string in contains: 
                folders = [f for f in folders if string.lower() in f.lower()]
            
            for string in does_not_contain: 
                folders = [f for f in folders if string.lower() not in f.lower()]
            
            if len(folders) == 0: 
                subfolders_absent.append(subfolder)
                
            elif len(folders) == 1: 
                subfolders_present.append(os.path.join(subfolder, folders[0]))
                
            elif len(folders) > 1: 
                print(f"\n{subfolder} has multiple subfolders with the specified name:")
                print(folders)
                subfolders_present.append(os.path.join(subfolder, folders[0]))
        
        if len(subfolders_present) == 0:
            print("\nNo folders contained the specified subfolders, therefore this step is not executed.")
        
        else:    
            print(f"\n{len(subfolders_present)} of {len(self.subfolders)} folders contained specified subfolder.")
            if print_absent: 
                print("\nFolders without subfolder")
                for s in subfolders_absent:
                    print("\t", s)

            self.walk_history.append(subfolders_present)
            self.walk_history_lost.append(subfolders_absent)
            self.subfolders = subfolders_present
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
        
            
    def up(self): 
        if len(self.walk_history) == 1:
            print("You can not go up as you are already at the subject folder")
        
        else: 
            self.subfolders = self.walk_history[-2]
            self.walk_history = self.walk_history[:-1]
            self.walk_history_lost = self.walk_history_lost[:-1]  
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
    
    def sample_down(self, i=0): 
        subfolder = self.subfolders[i]
        path = os.path.join(self.root, subfolder)
            
        folders, files = self.get_folders_files(path)
        print("\nCurrent path depth:", subfolder)
        print("Subfolders:")
        for s in folders: 
            print("\t", s)
    
    def current_path(self):
        return self.subfolders[0]
            
    def number_of_files(self, print_dict=False):
        
        dct = defaultdict(list)
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            _, files = self.get_folders_files(path)
            length = len(files)
            
            dct[str(length)].append(subfolder)
        
        for l, f in dct.items():
            print(f"\n{str(len(f))} folders contain {l} files each.")
            
        if print_dict: 
            for key, value in dct.items(): 
                print(f"Folders with {key} files:")
                for s in value: 
                    print("\t", s)

## Save content of folder as 3d numpy file

In [9]:
def save_3d_images(folder_class, save_to):
    for string in sorted(folder_class.subfolders):
        # get subject name and path
        subject, loc = string.split("/", 1)
        path = os.path.join(folder_class.root, string)
        
        # get all files in folder
        for root, folders, files in os.walk(path):
            break  
        files = sorted(files)
        
        # create 3d image
        three_d = []
        for image in files:
            if "DICOMDIR" not in image:
                ds = pydicom.dcmread(os.path.join(path, image))
                pix = ds.pixel_array
                three_d.append(pix)

        if len(three_d) != 0: 
            three_d = np.dstack(three_d)

            # create subfolder if not existent
            if not os.path.exists(save_to):
                os.makedirs(save_to)

            # save np array
            with open(f'{save_to}/{subject}.npy', 'wb') as f:
                np.save(f, three_d)

## Load data from own folders

In [1]:
class Project(object): 
    def __init__(self, project): 
        self.root = "/scratch/ptenkaate/Data"
        self.project = project
        
        self.m_path = os.path.join(self.root, self.project, "mask")
        self.p_path = os.path.join(self.root, self.project, "pcmra")
        
        if not os.path.exists(self.m_path):
            raise(Exception("This project does not exist, please specify a different project"))
        
        for root, folders, mask_files in os.walk(self.m_path):
            break   
        for root, folders, pcmra_files in os.walk(self.p_path):
            break   
        
        
        self.subjects = sorted([file for file in mask_files if file in pcmra_files])        
        self.subprojects = [project for i in range(len(self.subjects))]
        
#         print(self.subjects)
        no_pcmra = [file for file in mask_files if file not in pcmra_files]
        print("Masks without PCMRA:", no_pcmra)
        
        self.masks, self.masks_shape = self.get_files_in_folder(self.m_path, self.subjects)
        self.pcmras, self.pcmras_shape = self.get_files_in_folder(self.p_path, self.subjects)
        
        for subject, mask_shape, pcmra_shape in zip(self.subjects, self.masks_shape, self.pcmras_shape):
            if mask_shape != pcmra_shape: 
                print(f"Shapes of subject {subject} are not equal:")
                print(f"Mask shape: {mask_shape}, \t PCMRA shape: {pcmra_shape}")
                    
#         self.normalize()
        
        
    def get_files_in_folder(self, path, subjects): 
        files = []
        files_shape = []

        for subj in subjects: 
            with open(os.path.join(path, subj), "rb") as f:
                file = np.load(f)
                files.append(file)
                files_shape.append(file.shape)

        return files, files_shape
    
    
    def append_project(self, project):
        new_project = Project(project)
        
        if type(self.project) == str: 
            self.project = [self.project, project]
        else: 
            self.project.append(project)
        
        self.subjects += new_project.subjects
        self.subprojects += new_project.subprojects
        self.masks += new_project.masks
        self.masks_shape += new_project.masks_shape
        self.pcmras += new_project.pcmras
        self.pcmras_shape += new_project.pcmras_shape
                
    
    def filter_dimension_shape(self, dim, size, print_dropped=False): 
        subjects, subprojects, masks, masks_shape, pcmras, pcmras_shape = [], [], [], [], [], []
        for i in range(len(self.subjects)): 
            if self.masks_shape[i][dim] == size and self.pcmras_shape[i][dim] == size: 
                subjects.append(self.subjects[i])
                subprojects.append(self.subprojects[i])
                masks.append(self.masks[i])
                masks_shape.append(self.masks_shape[i])
                pcmras.append(self.pcmras[i])
                pcmras_shape.append(self.pcmras_shape[i])
            else: 
                if print_dropped:
                    print(f"Subject nr {i}, name {self.subjects[i]} does not have a size {size} at dimension {dim}")
        
        self.subjects = subjects
        self.subprojects = subprojects
        self.masks = masks
        self.masks_shape = masks_shape
        self.pcmras = pcmras
        self.pcmras_shape = pcmras_shape       
    
    
    def get_subject(self, i): 
        return self.subjects[i], self.pcmras[i], self.masks[i]
     
        
    def get_number_of_subjects(self):
        return len(self.masks)

    
    def get_number_of_slices(self):
        i = 0
        for mask in self.masks: 
            i += mask.shape[2]

        return i
    
    
    def normalize(self): 
        self.masks = [(mask - mask.min()) / (mask.max() - mask.min()) for mask in self.masks]
        self.pcmras = [(pcmra - pcmra.min()) / (pcmra.max() - pcmra.min()) for pcmra in self.pcmras]

## Show an image interactively

In [4]:
import sys
import numpy
    
# numpy.set_printoptions(threshold=sys.maxsize)
        
        
class Show_images(object):
    def __init__(self, suptitles, *images_titles):
        
        if type(suptitles) == str: 
            self.suptitles = []
            for i in range(images_titles[0][0].shape[2]): 
                self.suptitles.append(suptitles)
        else: 
            self.suptitles = suptitles
                    
        self.fig, self.ax = plt.subplots(1,len(images_titles))

        self.images = [x[0] for x in images_titles]
        self.titles = [x[1] for x in images_titles]
        
        rows, cols, self.slices = self.images[0].shape        
        self.ind = 0

        self.fig.suptitle(self.suptitles[self.ind])

        self.plots = []
        
        if self.images[0].shape[2] > 20: 
            self.ind = 10
        else:
            self.ind = self.images[0].shape[2] // 2
        
        if type(self.ax) == np.ndarray:
            pass
        else: 
            self.ax = np.array([self.ax])
            
        for (sub_ax, image, title) in zip(self.ax, self.images, self.titles): 
            sub_ax.set_title(title)
            plot = sub_ax.imshow(image[:, :, self.ind])
            self.plots.append(plot)

#         else: 
#             self.ax.set_title(self.titles[0])
#             plot = self.ax.imshow(self.images[0][:, :, self.ind])
#             self.plots.append(plot)
        
        self.plot_show = self.fig.canvas.mpl_connect('scroll_event', self.onscroll)
        
        figManager = plt.get_current_fig_manager()
        figManager.window.showMaximized()
        
        plt.show()

    def onscroll(self, event):
        if event.button == 'up':
            self.ind = (self.ind - 1) % self.slices
        else:
            self.ind = (self.ind + 1) % self.slices
        
        self.update()
        

    def update(self):
        
        self.fig.suptitle(self.suptitles[self.ind])
        
        for plot, image in zip(self.plots, self.images):
            plot.set_data(image[:, :, self.ind])
        
        self.ax[0].set_ylabel('Slice Number: %s' % self.ind)

        self.plots[0].axes.figure.canvas.draw()

In [ ]:
print("Imported classes.")